# Preperatory Blocks

#### Required Imports

Ready to do some programming? This very important assignment requires using the mitreattack package for Python to perform some TTP Analysis.

You can install the required package using the following command:
pip install mitreattack-python

Free Hint - This repository will be a great reference for this series of challenges
https://github.com/mitre-attack/mitreattack-python/tree/master/mitreattack/attackToExcel



In [2]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf

#### Input requested APTs

In [3]:
apt_list = ["APT28", "APT29", "Dragonfly", "Gamaredon Group", "Dragonfly", "Sandworm Team", "Turla", "Wizard Spider"]

#### Download and parse ATT&CK STIX data

In [10]:
attackdata = attackToExcel.get_stix_data("enterprise-attack") #functions = 'techniquesToDf, tacticsToDf, softwareToDf, groupsToDf,... https://github.com/mitre-attack/mitreattack-python/blob/master/mitreattack/attackToExcel/README.md
groups_data = stixToDf.groupsToDf(attackdata) #Keys = ['groups', 'associated software', 'techniques used', 'citations']

2023-07-24 19:36:30.590 | INFO     | mitreattack.attackToExcel.attackToExcel:get_stix_data:69 - Downloading ATT&CK data from github.com/mitre/cti
parsing relationships for type=group: 100%|██████████| 16530/16530 [00:00<00:00, 170895.63it/s]


# Program Blocks

## ASOM Dev

### Get TTP Data for APTs

Compile a list of techniques for each APT.

Provide how many techniqes are used by each APT.

In [20]:
group_techniques_df = groups_data["techniques used"] # Keys: ['source ID', 'source name', 'source type', 'mapping type', 'target ID', 'target name', 'target type', 'mapping description']


#print(group_techniques_df)|

apt_ttps = dict()
for apt in apt_list:
    data = group_techniques_df[group_techniques_df["source name"].str.contains(apt)]
    apt_ttps[apt] = data["target ID"].values


print(", ".join([str(len(apt_ttps[apt])) for apt in apt_ttps]))

86, 54, 56, 48, 67, 66, 46


### Identify the Most Common Techniques among the APTs

Lets do some analysis and identify what techniques are most common amount the APTs.

Provide a list of which techniques are associated with at least 6 of the 7 APTs (in numerical order)

Example:
['T1005', 'T1027', 'T1059.001', 'T1059.003']

Solution: ['T1005', 'T1027', 'T1059.001', 'T1059.003', 'T1070.004', 'T1071.001', 'T1083', 'T1105', 'T1204.002', 'T1547.001', 'T1566.001', 'T1588.002']

In [25]:
ttp_tracker = dict()
for apt in apt_ttps:
    for ttp in apt_ttps[apt]:
        if ttp in ttp_tracker:
            ttp_tracker[ttp].append(apt)
        else:
            ttp_tracker[ttp] = [apt]
            
ttp_common_sorted = sorted(ttp_tracker, key=lambda k: len(ttp_tracker[k]), reverse=True)

ttp_common_grouped = dict()

for ttp in ttp_tracker:
    count = len(ttp_tracker[ttp])
    if count in ttp_common_grouped:
        ttp_common_grouped[count].append(ttp)
    else: 
        ttp_common_grouped[count] = [ttp]
for count in sorted(ttp_common_grouped, reverse=True):
    print(f'{count}: {ttp_common_grouped[count]}')

print(f"Solution: {sorted(ttp_common_grouped[6] + ttp_common_grouped[7])}")

7: ['T1059.001']
6: ['T1070.004', 'T1105', 'T1204.002', 'T1547.001', 'T1566.001', 'T1588.002']
5: ['T1005', 'T1133', 'T1083', 'T1204.001', 'T1566.002', 'T1078', 'T1071.001', 'T1059.003']
4: ['T1140', 'T1583.001', 'T1203', 'T1036.005', 'T1003.003', 'T1021.002', 'T1595.002', 'T1505.003', 'T1562.001', 'T1053.005', 'T1047', 'T1087.002', 'T1112', 'T1018', 'T1033', 'T1027.010', 'T1082']
3: ['T1102.002', 'T1110', 'T1025', 'T1189', 'T1190', 'T1068', 'T1210', 'T1027', 'T1120', 'T1057', 'T1114.002', 'T1218.011', 'T1113', 'T1598.003', 'T1221', 'T1199', 'T1583.006', 'T1587.001', 'T1059.006', 'T1016', 'T1041', 'T1059.005', 'T1570']
2: ['T1098.002', 'T1560', 'T1560.001', 'T1119', 'T1070.001', 'T1078.004', 'T1213', 'T1039', 'T1586.002', 'T1564.003', 'T1056.001', 'T1003.001', 'T1074.001', 'T1071.003', 'T1036', 'T1090.003', 'T1040', 'T1110.001', 'T1110.003', 'T1027.001', 'T1218.005', 'T1546.003', 'T1591.002', 'T1195.002', 'T1069.002', 'T1135', 'T1012', 'T1021.001', 'T1003.002', 'T1584.004', 'T1485', 'T

### CCIR Selection

By this point, should hopefully have a list of all the techniques associated with the APTs. Which tactic has the most associated techniques from that list?

Free Hint - The following might be useful:
techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack") #Keys = ['techniques', 'procedure examples', 'associated mitigations', 'citations']

Provide the both the tactic T# and the number of associated TTPs as a comma seperated list

Example:
TA0005, 24


Solution: TA0005, 46

In [22]:
min_ttps = 0 #Select how common the TTPs must be in relation to the selects APTs. This will determine the resulting CCIRs and ultimately the length of the outputed ASOM.

In [23]:
techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack") #Keys = ['techniques', 'procedure examples', 'associated mitigations', 'citations']
techniques_df = techniques_data['techniques'] #Keys: ['ID', 'name', 'description', 'url', 'created', 'last modified',  'version', 'tactics', 'detection', 'platforms', 'data sources',  'is sub-technique', 'sub-technique of', 'defenses bypassed',  'contributors', 'permissions required', 'supports remote',  'system requirements', 'CAPEC ID', 'impact type',  'effective permissions', 'relationship citations'],

tactics_tracker = dict()
for count in ttp_common_grouped:
    if count >= min_ttps:
        for ttp in ttp_common_grouped[count]:
                technique = techniques_df[techniques_df["ID"].str.contains(ttp)]
                for tactic in technique['tactics'].values[0].split(', '):
                    if tactic in tactics_tracker:
                        tactics_tracker[tactic].append(ttp)
                    else:
                        tactics_tracker[tactic] = [ttp]

for tactic in tactics_tracker:
    print(f'CCIR: {tactic} - Indicators: {tactics_tracker[tactic]}')

tactic = sorted(tactics_tracker, key = lambda key: len(tactics_tracker[key]))[-1]
print(f'Solution: {tactic}, {len(tactics_tracker[tactic])}')

parsing relationships for type=technique: 100%|██████████| 16530/16530 [00:00<00:00, 126960.24it/s]


CCIR: Persistence - Indicators: ['T1098.002', 'T1078.004', 'T1546.003', 'T1078.002', 'T1547.004', 'T1542.003', 'T1546.015', 'T1037.001', 'T1137.002', 'T1546.008', 'T1136.003', 'T1098.005', 'T1556.007', 'T1098', 'T1136.001', 'T1137', 'T1136.002', 'T1078.003', 'T1546.013', 'T1543.003', 'T1133', 'T1078', 'T1505.003', 'T1053.005', 'T1547.001']
CCIR: Collection - Indicators: ['T1560', 'T1560.001', 'T1119', 'T1213', 'T1039', 'T1056.001', 'T1074.001', 'T1074.002', 'T1213.002', 'T1074', 'T1557.001', 'T1025', 'T1114.002', 'T1113', 'T1005']
CCIR: Defense Evasion - Indicators: ['T1070.001', 'T1078.004', 'T1564.003', 'T1036', 'T1027.001', 'T1218.005', 'T1078.002', 'T1055.001', 'T1550.001', 'T1542.003', 'T1211', 'T1564.001', 'T1550.002', 'T1014', 'T1070.006', 'T1134.001', 'T1548.002', 'T1027.006', 'T1556.007', 'T1553.005', 'T1550.003', 'T1027.002', 'T1562.004', 'T1564.002', 'T1027.004', 'T1553.006', 'T1134.002', 'T1027.011', 'T1027.005', 'T1078.003', 'T1055', 'T1553.002', 'T1036.004', 'T1222.001', 